In [1]:
import pandas as pd
import numpy as np
import math as math

In [2]:
truenews = pd.read_csv('true-split.csv')
fakenews = pd.read_csv('fake-split.csv')

In [3]:
truenews['True/Fake']='True'
fakenews['True/Fake']='Fake'

In [4]:
# gabung fake & true jadi 1
news = pd.concat([truenews, fakenews])
news["Article"] = news["title"] + news["text"]
news.sample(frac = 1)


,Unnamed: 0,title,text,subject,date,True/Fake,Article
468,23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake,How to Blow $700 Million: Al Jazeera America F...
383,21371,"As Syria war tightens, U.S. and Russia militar...","AL UDEID AIR BASE, Qatar (Reuters) - Even as t...",worldnews,"August 24, 2017",True,"As Syria war tightens, U.S. and Russia militar..."
30,23041,U.S. Inauguration: Historic Day Marks Beginnin...,"Mark Anderson American Free PressWASHINGTON,...",Middle-east,"January 21, 2017",Fake,U.S. Inauguration: Historic Day Marks Beginnin...
49,23060,Boiler Room #92 – The (Hollywood) Hills Have Eyes,Tune in to the Alternate Current Radio Network...,Middle-east,"January 13, 2017",Fake,Boiler Room #92 – The (Hollywood) Hills Have E...
158,21146,Vietnam says violations found at central bank ...,HANOI (Reuters) - Vietnam has found faults wit...,worldnews,"September 2, 2017",True,Vietnam says violations found at central bank ...
...,...,...,...,...,...,...,...
297,23308,DALLAS MAIDAN: Staged Snipers Designed to Infl...,"Daily Shooter 21st Century WireLast night, a h...",Middle-east,"July 8, 2016",Fake,DALLAS MAIDAN: Staged Snipers Designed to Infl...
185,21173,Al Shabaab bomb kills 12 in Somalia's Puntland,"BOSSASO, Somalia (Reuters) - An al Shabaab bom...",worldnews,"September 1, 2017",True,Al Shabaab bomb kills 12 in Somalia's Puntland...
403,21391,Romanian president opposes plans for judicial ...,BUCHAREST (Reuters) - Romania s justice minist...,worldnews,"August 23, 2017",True,Romanian president opposes plans for judicial ...
14,21002,Dangerous hurricane Irma moving towards Caribb...,"(Reuters) - Irma, now a category 4 , is headin...",worldnews,"September 5, 2017",True,Dangerous hurricane Irma moving towards Caribb...


In [5]:
#Data Cleaning
from nltk.corpus import stopwords
import string

In [6]:
def process_text(s):

    # cek tanda baca
    nopunc = [char for char in s if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_string = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_string

In [7]:

news['Clean Text'] = news['Article'].apply(process_text)

In [8]:
news.sample(5)

,Unnamed: 0,title,text,subject,date,True/Fake,Article,Clean Text
93,21081,"In shift, Merkel backs end to EU-Turkey member...",BERLIN (Reuters) - German Chancellor Angela Me...,worldnews,"September 3, 2017",True,"In shift, Merkel backs end to EU-Turkey member...","[shift, Merkel, backs, end, EUTurkey, membersh..."
335,21323,More than a thousand turn Philippine funeral t...,MANILA (Reuters) - More than a thousand people...,worldnews,"August 26, 2017",True,More than a thousand turn Philippine funeral t...,"[thousand, turn, Philippine, funeral, protest,..."
308,23319,MISSING DETAILS: Orlando Shooting 911 Transcri...,Shawn Helton 21st Century Wire The FBI have fi...,Middle-east,"June 22, 2016",Fake,MISSING DETAILS: Orlando Shooting 911 Transcri...,"[MISSING, DETAILS, Orlando, Shooting, 911, Tra..."
154,23165,‘COLOR REV’ AGIT PROP: George Soros MoveOn Agi...,Shawn Helton 21st Century WireAnti-Trump stree...,Middle-east,"November 11, 2016",Fake,‘COLOR REV’ AGIT PROP: George Soros MoveOn Agi...,"[‘COLOR, REV’, AGIT, PROP, George, Soros, Move..."
409,23420,"‘Vote All You Want, The Secret Government Won’...",21st Century Wire says Those who pull the stri...,Middle-east,"March 9, 2016",Fake,"‘Vote All You Want, The Secret Government Won’...","[‘Vote, Want, Secret, Government, Won’t, Chang..."


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
bow_transformer = CountVectorizer(analyzer=process_text).fit(news['Clean Text'])

print(len(bow_transformer.vocabulary_))

893


In [11]:

news_bow = bow_transformer.transform(news['Clean Text'])

In [12]:
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(899, 893)


In [14]:
#Train Naive Bayes Model
from sklearn.naive_bayes import MultinomialNB
fakenews_detect_model = MultinomialNB().fit(news_tfidf, news['True/Fake'])

In [15]:
#Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True

In [16]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))

              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       470
        True       1.00      1.00      1.00       429

    accuracy                           1.00       899
   macro avg       1.00      1.00      1.00       899
weighted avg       1.00      1.00      1.00       899



In [17]:
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(news['Article'], news['True/Fake'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

629 270 899


In [18]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)), 
    ('tfidf', TfidfTransformer()), 
    ('classifier', MultinomialNB()),
])
pipeline.fit(news_train,text_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x00000255103E2D38>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [19]:
predictions = pipeline.predict(news_test)

In [20]:
# Hasil Akhir
print(classification_report(predictions,text_test))

              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       145
        True       1.00      1.00      1.00       125

    accuracy                           1.00       270
   macro avg       1.00      1.00      1.00       270
weighted avg       1.00      1.00      1.00       270

